In [13]:
import numpy as np 
import cv2
from PIL import Image
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'c:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

* cv2 : opencv library for computer vision and image processing tasks.
* PIL : Python Imaging Library (Pillow fork) to handle image files.
* pytesseract :  a Python wrapper for the Tesseract OCR engine.
* tess.pytesseract.tesseract_cmd : Sets the path to the Tesseract executable file. So it  must specify this if Tesseract isn't in the system's PATH variable. (Windows)

In [14]:
def ratioCheck(area, width, height):
    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio
    if (area < 1063.62 or area > 73862.51) or (ratio < 3 or ratio > 6):
        return False
    return True

This function ----> filter shapes or objects based on size and proportions.
* Calculates the aspect ratio as width/height (ensures it's always ≥ 1).
* Checks the area is within the range [1063.62, 73862.51]
* Checks the aspect ratio is within the range [3, 6].

In [15]:
def isMaxWhite(plate):
    avg = np.mean(plate)
    if (avg >= 115):
        return True
    else:
        return False

This function ----> ensure the plate has sufficient brightness for further processing or analysis.
* Calculates the average pixel intensity of the image plate.
* Compares the average intensity to 115

In [16]:
def ration_and_rotation(rect):
    (x, y), (width, height), rect_angle = rect
    if (width > height):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle
    if angle > 15:
        return False 
    if height == 0 or width == 0:
        return False
    area = height * width
    if not ratioCheck(area, width, height):
        return False 
    else:
        return True 

This function ----> ensures rectangles conform to specific shape and orientation requirements for further processing.
* Unpacks rectangle attributes
* Adjusts the angle based on the rectangle's orientation
* Checks the angle constraint
* Checks for valid dimensions
* Calculates area and validates it using the ratioCheck function

In [17]:
def clean2_plate(plate):
    gray_img = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray_img, 110, 255, cv2.THRESH_BINARY)
    if cv2.waitKey(0) & 0xff == ord('q'):
        pass 
    num_contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if num_contours:
        contour_area = [cv2.contourArea(c) for c in num_contours]
        max_cntr_index = np.argmax(contour_area)

        max_cnt = num_contours[max_cntr_index]
        max_cntArea = contour_area[max_cntr_index]
        x, y, w, h = cv2.boundingRect(max_cnt)

        if not ratioCheck(max_cntArea, w, h):
            return plate, None 
    
        final_img = thresh[y:y+h, x:x+w]
        return final_img, [x, y, w, h]
    else:
        return plate, None 

This function ---->  is useful for isolating the most significant object in an the plate, based on contours and shape checks.
* Convert to grayscale
* Thresholding
* Wait for key press
* Find contours
* Identify the largest contour
* Check aspect ratio and area
* Extract the region of interest (ROI)  
  Returns the cropped image (final_img) along with its bounding box coordinates
  

In [18]:
img = cv2.imread("testData/sample1.jpg")
print('Number input image ...')
cv2.imshow("input ", img)

if cv2.waitKey(0) & 0xff == ord('q'):
    cv2.destroyAllWindows()  # Close the image window

img2 = cv2.GaussianBlur(img, (3, 3), 0)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

img2 = cv2.Sobel(img2, cv2.CV_8U, 1, 0, ksize=3)
_, img2 = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
morph_img_threshold = img2.copy()
cv2.morphologyEx(src=img2, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
num_contours, hierarchy = cv2.findContours(morph_img_threshold, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cv2.drawContours(img2, num_contours, -1, (0, 255, 0), 1)

for i, cnt in enumerate(num_contours):
    min_rect = cv2.minAreaRect(cnt)

    if ration_and_rotation(min_rect):
        x, y, w, h = cv2.boundingRect(cnt)
        plate_img = img[y:y + h, x:x + w]
        print('Number identified number plate ...')
        cv2.imshow('num plate image', plate_img)
        if cv2.waitKey(0) & 0xff == ord('q'):
            cv2.destroyAllWindows()  # Close the image window
        if isMaxWhite(plate_img):
            clean_plate, rect = clean2_plate(plate_img)
            if rect:
                x1, y1, w1, h1 = rect
                x, y, w, h = x + x1, y + y1, w1, h1
                plate_im = Image.fromarray(clean_plate)
                text = tess.image_to_string(plate_im, lang='eng')
                print('Number Detected Plate Text : ', text)


Number input image ...
Number identified number plate ...
Number Detected Plate Text :  ‘PL8 REC


Number identified number plate ...
Number identified number plate ...


--> Extracts the license plate from an image and uses Tesseract to read the text on the plate.  
* Read and display the input image  
* Image preprocessing:  
-) Gaussian Blur : to smooth the image  
-) Convert to Grayscale  
-) Sobel Edge Detection : detects edges in the horizontal direction   
-) Thresholding : Otsu's binarization to separate the foreground and background.  
* Morphological operation (Closing)  
-)  creates a rectangular structuring element for morphological operations.  
-) performs a morphological closing operation to fill small gaps in the binary image.  
* Contour detection  
-) finds the contours in the thresholded image.  
-)  draws the contours on the image.  
* Process each contour:  
-) For each contour, it calculates the minimum bounding rectangle using cv2.minAreaRect(cnt).  
-) Aspect ratio and rotation check  
-) filters out invalid rectangles based on the aspect ratio and rotation angle.  
-) Extract the plate region: :  If the contour passes the checks, cv2.boundingRect(cnt) is used to get the bounding box coordinates and extract the plate image from img.  
* Display the extracted plate  
-) Waits for the user to press 'q' to close the window.  
* Check if the plate is sufficiently white:  
-) isMaxWhite(plate_img) checks if the plate image has a high average brightness (suggesting it is a valid plate).  
-) Clean the plate: clean_plate, rect = clean2_plate(plate_img) processes the plate further to remove noise or irrelevant regions.  
* Extract text from the cleaned plate:  
-) If a valid rectangle (rect) is found, the text is extracted using Tesseract OCR:  
